In [10]:
import pandas as pd
import altair

# Data Extraction

In [ ]:
from dotenv import load_dotenv
import os
from Crypto.Hash import MD5
import requests
import time

In [55]:
class DataExtractor:

    __url:str = "http://gateway.marvel.com/v1/public/"
    #__endpoints:str = ["characters", "comics", "creators", "events", "series", "stories"]
    __endpoints:str = ["characters", "comics"]
    __priv_key:str = None
    __pub_key:str = None

    def __init__(self):  
        """
        Loads the public and private keys from the .env file, necessary to acess the API.
        """

        load_dotenv()
        self.__priv_key = os.getenv('PRIV_KEY')
        self.__pub_key = os.getenv('PUB_KEY')

    def __format_request_url(self, endpoint:str) -> str:
        """
        Formats the URL to be used in a request, using the timestamp, public and private keys and the desired endpoint.

        Args:
            endpoint (str): The endpoint of the request URL.

        Returns:
            str: The formatted request URL.

        Raises:
            ValueError: If the provided endpoint is not valid.
        """

        # Verifies the endpoint exists in the API
        if endpoint not in self.__endpoints:
            raise ValueError(f'{endpoint} is not a valid endpoint.')

        timestamp = time.time()
        timestamp_str = f"ts={timestamp}"

        apikey_str = "apikey=" + self.__pub_key

        # Calculates the MD5 hash, needed for the URL
        md5_message = str(timestamp) + self.__priv_key + self.__pub_key
        hash_str = "hash=" + MD5.new(str.encode(md5_message)).hexdigest()

        request_url = self.__url + endpoint + "?" + timestamp_str + "&" + apikey_str + "&" + hash_str
        return request_url
    
    def __make_request(self, request_url:str) -> dict:
        """
        Makes a request to the API.

        Args:
            request_url (str): The request URL.

        Returns:
            dict: The data of the response in JSON.

        Raises:
            requests.HTTPError: If the received status code is not 200.
        """

        # Makes the request
        response = requests.get(request_url)

        if response.status_code != 200:
            raise requests.HTTPError(f"Received status code {response.status_code} for {response.url}")
        
        # Filters metadata out
        return response.json()['data']['results']

    def extract(self) -> dict:
        """
        Extracts the data from all API endpoints.

        Args:

        Returns:
            dict: The API data, where the key-value pairs are the endpoints (str) and its 
            data (dict) in JSON format, if the request was successful, and None otherwise.

        Raises:
        """

        api_data = {}

        # For each endpoints, tries to extract the data and add it to the api_data dictionary
        for endpoint in self.__endpoints:
            try:
                request_url = self.__format_request_url(endpoint)
                endpoint_data = self.__make_request(request_url)
                api_data[endpoint] = endpoint_data

            except Exception as e:
                print(f"Error: {e}")

                # If the request was unsuccessful, adds the endpoint key to the dict with null data
                api_data[endpoint] = None 
        
        return api_data

In [ ]:
de = DataExtractor()
data_json = de.extract()

# Data Transformation

In [8]:
class dataTransformator:

    def parse_topic(topic_data:dict) -> pd.DataFrame:
        pass

    def parse(data:dict) -> list[pd.DataFrame]:
        pass

# Data analysis